In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
trip = validation_data_input.get_data('trip')
hh = validation_data_input.get_data('household')
person = validation_data_input.get_data('person')

In [3]:
# parcel land use data
df_parcel = pd.read_csv(os.path.join(config['model_dir'],r'outputs\landuse\buffered_parcels.txt'),
                        delim_whitespace=True,
                        usecols=['parcelid','emptot_1','hh_1'])

In [4]:
df_trip = trip.copy()
df_hh = hh.copy()
df_person = person.copy()

# Add column for (potential) drivers adults (all hh members 16 and above)
df_hh['drivers'] = df_hh['hhsize']-df_hh['hh515']-df_hh['hhcu5']#-df_hh['hhhsc']
# auto_ownership with 4+
df_hh['auto_ownership_4+'] = df_hh['hhvehs'].apply(lambda x: "4+" if x>=4.0 else str(x))
# hhsize with 4+
df_hh['hhsize_4+'] = df_hh['hhsize'].apply(lambda x: "4+" if x>=4.0 else str(x))
# auto availability
df_hh['auto_count_driver'] = df_hh['hhvehs']-df_hh['drivers']
df_hh['auto_count_worker'] = df_hh['hhvehs']-df_hh['hhwkrs']
df_hh['auto_available_driver'] = np.where(df_hh['drivers']<=0, "no driver",
                                          np.where(df_hh['hhvehs']<=0, "no car",
                                                   np.where(df_hh['auto_count_driver']<0, "cars fewer than drivers", "enough cars")))

df_hh['auto_available_worker'] = np.where(df_hh['hhwkrs']<=0, "no worker",
                                          np.where(df_hh['hhvehs']<=0, "no car",
                                                   np.where(df_hh['auto_count_worker']<0, "cars fewer than workers", "enough cars")))


# add person type labels
ptype_cat = {1: "full time worker",
             2: "part time worker",
             3: "non-worker age 65+",
             4: "other non-working adult",
             5: "university student",
             6: "grade school student/child age 16+",
             7: "child age 5-15",
             8: "child age 0-4"}
df_person['pptyp_label'] = df_person['pptyp'].map(ptype_cat)

mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: transit",
            8: "8: school bus",
            9: "9: tnc"}
df_trip['mode_label'] = df_trip['mode'].map(mode_cat)

dpurp_cat = {1: "Work",
              2: "School",
              3: "Escort",
              4: "Personal Business",
              5: "Shop",
              6: "Meal",
              7: "Social"}
df_trip['dpurp_label'] = df_trip['dpurp'].map(dpurp_cat)

df_hh = df_hh.merge(df_parcel,how="left",left_on='hhno',right_on='parcelid')
df_person = df_person.merge(df_hh,
                          how='left', on=['hhno','source']) # get auto ownership from hh data

df_trip = df_trip.merge(df_person, how='left', on=['pno','hhno','source'])

In [5]:
hb_trip = df_trip.loc[~(df_trip['dpurp'].isin([1,2,3]))].copy()

In [6]:
df_plot = hb_trip.groupby(['source','mode_label'])['trexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trexpfac']. \
    apply(lambda x: x / float(x.sum()))

df_plot_ct = hb_trip.groupby(['source','mode_label'])['trexpfac'].count().reset_index(). \
    rename(columns={'trexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','mode_label'])

fig = px.bar(df_plot.sort_values(by=['source']), x="mode_label", y="percentage", color="source",
             barmode="group",hover_data=['sample count'],title="other home-based trip mode")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  xaxis = dict(dtick = 1, categoryorder='category ascending'),
                  yaxis=dict(tickformat=".2%"))
fig.show()

In [7]:
df_plot = hb_trip.groupby(['source','dpurp_label','mode_label'])['trexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','dpurp_label'], group_keys=False)['trexpfac']. \
    apply(lambda x: x / float(x.sum()))

df_plot_ct = hb_trip.groupby(['source','dpurp_label','mode_label'])['trexpfac'].count().reset_index(). \
    rename(columns={'trexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','dpurp_label','mode_label'])

fig = px.bar(df_plot.sort_values(['mode_label']),
             x="percentage", y="mode_label", color="source",barmode="group",
             facet_col='dpurp_label', facet_col_wrap=1, orientation='h',
             hover_data=['sample count'],
             category_orders={"mode_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: transit",
                                               "8: school bus","9: other–survey only"]},
             title="other home-based trip mode choice")
fig.update_layout(height=1000, width=650)
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_xaxis(lambda a: a.update(tickformat = ".1%"))
fig.show()

### mode choice by segment

In [8]:
def plot_mode_choice(df: pd.DataFrame, grp_var: str, order_list: dict, title_name: str, n_nol: int, height=400, width=800):
    df_plot = df.groupby(['source',grp_var,'mode_label'])['trexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['trexpfac']. \
        apply(lambda x: x / float(x.sum()))

    df_plot_ct = df.groupby(['source',grp_var,'mode_label'])['trexpfac'].count().reset_index(). \
        rename(columns={'trexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',grp_var,'mode_label'])

    fig = px.bar(df_plot.sort_values(['mode_label']),
                 x="percentage", y="mode_label", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol, orientation='h',
                 hover_data=['sample count'],
                 category_orders=order_list,
                 title="other home-based trip mode choice by " + title_name)
    fig.update_layout(height=height, width=width)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.for_each_xaxis(lambda a: a.update(tickformat = ".1%"))
    fig.show()

In [9]:
plot_mode_choice(hb_trip,"pptyp_label",
                 {"pptyp_label":["full time worker","part time worker","non-worker age 65+","other non-working adult",
                                 "university student","grade school student/child age 16+","child age 5-15","child age 0-4"],
                  "mode_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: transit",
                                   "8: school bus","9: other–survey only"]},
                 "person type",2,1000)

In [10]:
plot_mode_choice(hb_trip,"hhsize_4+",
                 {"hhsize_4+":["1","2","3","4+"],
                  "mode_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: transit",
                                   "8: school bus","9: other–survey only"]},
                 "household size",2,600)

In [11]:
plot_mode_choice(hb_trip.loc[hb_trip['auto_ownership_4+']!="-1"],"auto_ownership_4+",
                 {"auto_ownership_4+":["0","1","2","3","4+"],
                  "mode_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: transit",
                                   "8: school bus","9: other–survey only"]},
                 "auto ownership",2,800)

In [12]:
plot_mode_choice(hb_trip.loc[hb_trip['auto_available_driver'] != "no dirver"], "auto_available_driver",
                 {"auto_available_driver": ["no car", "cars fewer than drivers", "enough cars"],
                  "mode_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: transit",
                                   "8: school bus","9: other–survey only"]},
                 "auto availability (driver, showing only households with at least one driver)", 3, 500, 1000)